In [8]:
# this notebook preprocessed six tables downloaded from USPTO (merged to one file)
# also it created the dictionary for further matching of company names
import pandas as pd
import numpy as np
import dill

In [4]:
assignee = pd.read_pickle('preprocess files/assignee')

In [6]:
assignment = pd.read_pickle('preprocess files/assignment')
assignor = pd.read_pickle('preprocess files/assignor')
docid = pd.read_pickle('preprocess files/docid')
merge_1 = assignee.merge(assignment, how='outer', on='rf_id')
merge_2 = merge_1.merge(assignor, how='outer', on='rf_id')
merge_3 = merge_2.merge(docid, how = 'outer', on='rf_id')

In [9]:
dill.dump(merge_3, open('merge_3.pkd', 'wb'))

In [ ]:
# ee_name = merge_3['ee_name']
# ee_name_uni = list(set(list(ee_name)))
# ee_name_clean = [str(name) for name in ee_name_uni if str(name)!= 'nan']
# ee_name_sorted = sorted(ee_name_clean)
# pd.Series(ee_name_sorted).to_csv('ee_names.csv')
# companylist0 = pd.read_csv('companylist_0.csv', usecols=['Symbol','Name','Sector','Industry'])
# companylist1 = pd.read_csv('companylist_1.csv', usecols=['Symbol','Name','Sector','Industry'])
# companylist2 = pd.read_csv('companylist_2.csv', usecols=['Ticker','Name','Sector','Industry'])
# companylist2.rename(columns={'Ticker':'Symbol'}, inplace=True)
# companylist = (companylist0.append(companylist1).append(companylist2))
# companylist.to_csv('companylist.csv')
# len(companylist)

In [316]:
companylist = pd.read_csv('companylist.csv', index_col=0)
ee_names = pd.read_csv('ee_names.csv', index_col=0, header=None)
or_names = pd.read_csv('or_names.csv', index_col=0, header=None)

/Users/annzhong/anaconda/lib/python3.6/site-packages/numpy/lib/arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
# assignor = pd.read_pickle('assignor')
# or_name_uni = list(set(list(assignor['or_name'])))
# or_name_clean = [str(name) for name in or_name_uni if str(name)!= 'nan']
# or_name_sorted = sorted(or_name_clean)
# pd.Series(or_name_sorted).to_csv('or_names.csv')

In [ ]:
# import spacy
# nlp = spacy.load('en')

In [283]:
# ee_name_arr = np.array(ee_names[1])
# ee_org = []
# for ee_name in ee_name_arr:
#     doc = nlp(ee_name)

#     if len(doc.ents)>0:
#         for ent in doc.ents:
#             ee_org.append(str(ent))
#             break
#     else:
#         ee_org.append('no ents available!')

# ee_names['filtered'] = ee_org
# ee_names.to_csv('ee_names_2.csv')

In [323]:
patent_names = ee_names.append(or_names)
patent_names_list = sorted(list(set(list(patent_names[1]))))

In [368]:
companylist_no_none = [name.upper().replace('-',' ').replace('.','').replace(',','')
                       for name in list(companylist['Name']) if str(name) != 'nan']
companylist_sorted = sorted(list(set(companylist_no_none)))

In [374]:
patent_name_list2 = [name.replace('-',' ').replace('.','').replace(',','')\
                     .replace(' & ', '&').replace('  ',' ').replace('\"','').strip() 
                     for name in patent_names_list]
patentname_sorted = sorted(list(set(patent_name_list2)))

In [377]:
# patentname_sorted

In [395]:
companylist_sorted = [name for name in companylist_sorted if len(name)>1]

In [396]:
# now match patentname_sorted with companylist_sorted, from which can get the ticker name

# first categorize companylist_sorted into A-Z for faster matching later
import re

match_dict = {}
curr_letter = ' '

for i in patentname_sorted:
    letter = list(i)[0]
    if curr_letter != letter:
        data = [re.compile(item) for item in companylist_sorted if list(item)[0] == letter]
        if data == []:
            pass
        else:
            match_dict[letter] = data
            curr_letter = letter
    elif curr_letter == letter:
        continue

In [397]:
# match_dict
# remove those name that doesn't have in database
patent_name_filter = [name for name in patentname_sorted 
                      if list(name)[0] in list(match_dict.keys())]
filtered = sorted(list(set(patent_name_filter)))

In [398]:
filtered = [name for name in filtered if len(name)>1]

In [399]:
company_dict = {}
no_match = []

for i in filtered:
    key = list(i)[0]
    dictionary = match_dict[key]
    for r in dictionary:
        if r.match(i):
            company_dict[i] = r.match(i).group(0)
        else:
            no_match.append(i)

In [410]:
# company_dict

In [401]:
import dill

In [404]:
dill.dump(company_dict, open('company_dict.pkd', 'wb'))